In [1]:
def weighted_average(value_list, weight_list):
    result = sum([value_list[i]*weight_list[i] for i in range(len(value_list))])/sum(weight_list)
    return result

In [2]:
# To Do: ommit retrning municipality_pop_book

In [3]:
def read_population_data_and_sum_in_municipality_level(datapath, separator, text_encoding_format, number_of_coloumns, measure_coloumn):
    ### aim: read the zipcode level data downloaded from statistics Finalnd and return population and average coordinate of each municipality 
    #data heading is: 
    #"Postal code area","X coordinate in metres","Y coordinate in metres","Surface area","Inhabitants, total, 2018 (HE)","Males, 2018 (HE)","Females, 2018 (HE)","Average age of inhabitants, 2018 (HE)","0-2 years, 2018 (HE)","3-6 years, 2018 (HE)","7-12 years, 2018 (HE)","13-15 years, 2018 (HE)","16-17 years, 2018 (HE)","18-19 years, 2018 (HE)","20-24 years, 2018 (HE)","25-29 years, 2018 (HE)","30-34 years, 2018 (HE)","35-39 years, 2018 (HE)","40-44 years, 2018 (HE)","45-49 years, 2018 (HE)","50-54 years, 2018 (HE)","55-59 years, 2018 (HE)","60-64 years, 2018 (HE)","65-69 years, 2018 (HE)","70-74 years, 2018 (HE)","75-79 years, 2018 (HE)","80-84 years, 2018 (HE)","85 years or over, 2018 (HE)","Aged 18 or over, total, 2018 (KO)"
    #an example line from data:
    #00130 Kaartinkaupunki (Helsinki),386244,6671474,428960,1523,735,788,41,41,47,63,29,24,26,98,160,139,122,116,103,120,101,79,79,88,47,26,15,1319
    line_counter = 0
    municipality_info_book = {}
    municipality_pop_book = {}
    municipality_coordinates_book = {}
    with open(data_path, 'r', encoding=text_encoding_format) as data:
        for line in data:
            line_counter += 1
            if line_counter > 4:
                fields = line.strip().split(separator)
                if len(fields) == number_of_coloumns:
                    area_name_string = fields[0]
                    municipality = area_name_string.strip().split(' ')[-1]
                    x_coordinate = int(fields[1])
                    y_coordinate = int(fields[2])
                    population = int(fields[measure_coloumn])
                    if municipality not in municipality_info_book.keys():
                        municipality_info_book[municipality] = []
                    municipality_info_book[municipality].append((population, x_coordinate, y_coordinate))
 
        for municipality in municipality_info_book.keys():
            #calculate the population sum in each municipality an coordinates of population center (weighted average of 
            pop_list, x_list, y_list = zip(*municipality_info_book[municipality])
            pop_sum = sum(pop_list)
            x_avg = weighted_average(x_list, pop_list)
            y_avg = weighted_average(y_list, pop_list)
            municipality_pop_book[municipality] = pop_sum
            municipality_coordinates_book[municipality] = (x_avg, y_avg)        
    return municipality_pop_book, municipality_coordinates_book

In [4]:
### Reading the data and extracting poulation to calculate the center of population of each municipality
#data_path = "/m/cs/scratch/networks/heydars1/population_flow/data/population_data/population_2017_cleaned.csv"
data_path = "./data/population_data/population_2017_cleaned.csv"
## this function can be used to extrct population and coordinates info for "municipalities". In that case, population is sum of all the zipcode regions in the municipality and x ant y coordinate are coordinates of center of population of the municipality (weighted average)
_ , coordinates_book = read_population_data_and_sum_in_municipality_level(data_path, ';', 'iso-8859-1', 5, 3) #notice that in the population data there is a municipality named "Valtimo" which can not be found in communting data      

In [5]:
len(coordinates_book)

310

In [6]:
import pickle
path_for_saving_population_centres_coordinates = "./data/population_data/centre_of_populations_of_municiplaities.pkl"
pickle.dump(coordinates_book, open(path_for_saving_population_centres_coordinates , "wb"))

----

In [13]:
import os
import geopandas as gpd

In [9]:
dir_path = os.path.abspath('')
data_path = os.path.join(dir_path, 'data')

In [14]:
def read_commutters_data(data_path, text_encoding_format, number_of_header_lines,
                        number_of_coloumns, measure_coloumn):
    measure_book = {}
    line_counter = 0
    aland_set = {'Brändö','Eckerö','Finström','Föglö','Geta','Hammarland','Jomala','Kumlinge','Kökar','Lemland','Lumparland','Maarianhamina - Mariehamn','Mariehamn','Saltvik','Sottunga','Sund','Vårdö'}
    with open(data_path, 'r', encoding=text_encoding_format) as data:
        for line in data:
            line_counter += 1
            if line_counter > number_of_header_lines:
                fields = line.strip().split(';')
                assert len(fields) == number_of_coloumns
                city = fields[0]
                if city not in aland_set:
                    measure = int(fields[measure_coloumn])
                    measure_book[city] = measure
    return measure_book

In [15]:
tessellation = gpd.read_file('./data/finland-municipalities-topojson.geojson').rename(columns={'id': 'tile_ID'})

dict_change_name = {
    'Inkoo':'Ingå',
    'Koski Tl':'Koski_Tl',
    'Kristiinankaupunki':'Kristinestad',
    'Kruunupyy':'Kronoby',
    'Luoto':'Larsmo',
    'Maalahti':'Malax',
    'Maarianhamina':'Mariehamn',
    'Mustasaari':'Korsholm',
    'Närpiö':'Närpes',
    'Pietarsaari':'Jakobstad',
    'Pedersören kunta':'Pedersöre',
    'Uusikaarlepyy':'Nykarleby',
    'Kemiönsaari':'Kimitoön',
    'Parainen':'Pargas'
}

tessellation['name'].replace(dict_change_name, inplace=True)

# epsg:3067 ETRS89 / TM35FIN(E,N)
muni_pcrs = tessellation.to_crs(3067)
muni_pcrs['centroid'] = muni_pcrs.geometry.centroid

In [16]:
### Reading the data on center of population of each municipality
coordinates_book = pickle.load( open( dir_path + "/data/population_data/centre_of_populations_of_municiplaities.pkl", "rb" ) )
print(len(coordinates_book))
aland_set = {'Brändö','Eckerö','Finström','Föglö','Geta','Hammarland','Jomala','Kumlinge','Kökar','Lemland','Lumparland','Maarianhamina - Mariehamn','Mariehamn','Saltvik','Sottunga','Sund','Vårdö'}
entries_to_remove = aland_set
for m in entries_to_remove:
    coordinates_book.pop(m, None)
    
coordinates_book['Valtimo'] = muni_pcrs[muni_pcrs['name'] == 'Valtimo'].centroid.geometry.values[0].coords[0]
print(len(coordinates_book))

310
295


In [17]:
path_for_saving_population_centres_coordinates = "./data/population_data/centre_of_populations_of_municiplaities.pkl"
pickle.dump(coordinates_book, open(path_for_saving_population_centres_coordinates , "wb"))